In [1]:
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.utils.misc import Timer
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import glob
import os

timer = Timer()

In [2]:
# MobileNet SSD # --- Le Anh trained ---
net_type = "mb1-ssd"
# net_type = "mb2-ssd-lite"

# model_path = "models/mb1-ssd-Epoch-19-Loss-6.089628639675322.pth"
model_path = "models/mb1-ssd-Epoch-105-Loss-inf.pth"
# model_path = "models/mb2-ssd-lite-mp-0_686.pth"

# label_path = "models/voc-model-labels-orig.txt"
label_path = "models/voc-model-labels.txt"
# --------------- #

class_names = [name.strip() for name in open(label_path).readlines()]
num_classes = len(class_names)
if net_type == 'vgg16-ssd':
	net = create_vgg_ssd(len(class_names), is_test=True)
elif net_type == 'mb1-ssd':
	net = create_mobilenetv1_ssd(len(class_names), is_test=True)
elif net_type == 'mb1-ssd-lite':
	net = create_mobilenetv1_ssd_lite(len(class_names), is_test=True)
elif net_type == 'mb2-ssd-lite':
	net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)
elif net_type == 'sq-ssd-lite':
	net = create_squeezenet_ssd_lite(len(class_names), is_test=True)
else:
	print("The net type is wrong. It should be one of vgg16-ssd, mb1-ssd and mb1-ssd-lite.")
	sys.exit(1)
	
net.load(model_path)

if net_type == 'vgg16-ssd':
	predictor = create_vgg_ssd_predictor(net, candidate_size=200)
elif net_type == 'mb1-ssd':
	predictor = create_mobilenetv1_ssd_predictor(net, candidate_size=200)
elif net_type == 'mb1-ssd-lite':
	predictor = create_mobilenetv1_ssd_lite_predictor(net, candidate_size=200)
elif net_type == 'mb2-ssd-lite':
	predictor = create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200)
elif net_type == 'sq-ssd-lite':
	predictor = create_squeezenet_ssd_lite_predictor(net, candidate_size=200)
else:
	print("The net type is wrong. It should be one of vgg16-ssd, mb1-ssd and mb1-ssd-lite.")
	sys.exit(1)

print("Loading Trained Model is Done!\n")

print("Starting Detection...\n")

Loading Trained Model is Done!

Starting Detection...



In [12]:
img_src = glob.glob("./images/*.jpg")
print(img_src)

['./images/bdd_test_5.jpg', './images/bdd_test_2.jpg', './images/bdd_test_7.jpg', './images/bdd_test_3.jpg', './images/bdd_test_8.jpg', './images/bdd_test_6.jpg', './images/bdd_test_4.jpg', './images/bdd_test_1.jpg']


In [32]:
print(class_names)

['BACKGROUND', 'train', 'truck', 'traffic light', 'traffic sign', 'rider', 'person', 'bus', 'bike', 'car', 'motor']


In [40]:
label_names = ["BACKGROUND", "train", "truck", "traffic_light", "traffic_sign",
               "rider", "person", "bus", "bike", "car", "motor"]

In [41]:
for img_src_i in img_src:
#     img_name = 'bdd_test_6'
    img_name = os.path.splitext(os.path.basename(img_src_i))[0]
    print(img_name)    
    
    orig_image = cv2.imread(img_src_i)

    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)

    color = np.random.uniform(0, 255, size = (10, 3))

    timer.start()
    boxes, labels, probs = predictor.predict(image, 10, 0.4)
    interval = timer.end()
    print('Time: {:.2f}s, Detect Objects: {:d}.'.format(interval, labels.size(0)))

    fps = 1/interval
    
    # Create and write out a text file:
    text_file = open("./texts/" + img_name + ".txt", "w+")

    for i in range(boxes.size(0)):
        box = boxes[i, :]
        label = f"{class_names[labels[i]]}: {probs[i]:.2f}"

        i_color = int(labels[i])
        # color = (25*(i+1), 25*(i+1), 25*(i+1))
        # cv2.rectangle(orig_image, (box[0], box[1]), (box[2], box[3]), (labels[i]*25, labels[i]*25, labels[i]*25), 2)
        cv2.rectangle(orig_image, (box[0], box[1]), (box[2], box[3]), color[i_color], 2)

        cv2.putText(orig_image, label,
                    (box[0] - 10, box[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,  # font scale
                    color[i_color],
                    2)  # line type

        cv2.putText(orig_image, "FPS = " + str(int(fps)),
                    (1080, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        print(f"{label_names[labels[i]]}", f"{probs[i]:.6f}", 
              int(box[0]), int(box[1]), int(box[2]), int(box[3]), 
              file=text_file)
    
    text_file.close()

print("Image Shape: ", orig_image.shape, ". Check the result!")

bdd_test_5
Inference time:  0.03354835510253906
Time: 0.09s, Detect Objects: 4.
bdd_test_2
Inference time:  0.03724956512451172
Time: 0.08s, Detect Objects: 4.
bdd_test_7
Inference time:  0.030572175979614258
Time: 0.07s, Detect Objects: 3.
bdd_test_3
Inference time:  0.03054046630859375
Time: 0.07s, Detect Objects: 6.
bdd_test_8
Inference time:  0.031526803970336914
Time: 0.07s, Detect Objects: 7.
bdd_test_6
Inference time:  0.033592939376831055
Time: 0.07s, Detect Objects: 6.
bdd_test_4
Inference time:  0.02860879898071289
Time: 0.07s, Detect Objects: 4.
bdd_test_1
Inference time:  0.03197956085205078
Time: 0.07s, Detect Objects: 6.
Image Shape:  (720, 1280, 3) . Check the result!
